In [1]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5


import sys
import os
import json
import functools
import boto3
import numpy as np
import pandas as pd
from datetime import datetime
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, FloatType, DateType
import logging







import sys
import json
import boto3
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
 
glueContext._jsc.hadoopConfiguration().set("fs.s3.useRequesterPaysHeader","true") ## this is needed for permissions
spark._jsc.hadoopConfiguration().set("fs.s3.useRequesterPaysHeader","true") ## this is needed for permissions
 
spark = glueContext.spark_session
spark.catalog.clearCache()
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 749b0ce0-d9de-44cd-95f4-f500ca1190d3
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session 749b0ce0-d9de-44cd-95f4-f500ca1190d3 to get into ready status...
Session 749b0ce0-d9de-44cd-95f4-f500ca1190d3 ha

In [2]:
# -------- AWS S3 Configuration -------- #
# Initialize the S3 client for data access
s3_client = boto3.client('s3')

In [3]:
# Define your S3 bucket name and the file path within the bucket
bucket_name = 'cci-dig-aicoe-data-sb'
file_key = 'processed/containment_metric/containment_metric_config_json/containment_metric.json'

In [4]:
read_json = s3_client.get_object(Bucket=bucket_name, Key=file_key)

# Load the file content into the config_data variable
config_data = json.loads(read_json['Body'].read().decode('utf-8'))
print(config_data)

{'features': [{'feature_name': 'SMART HELP', 'feature_id': '15', 'metrics': [{'Primary_intent': 'Equipment Support', 'Primary_intent_detail': 'SmartHelp', 'cont_display_metric_name': 'Smarthelp_containment_rate', 'cont_metric_seq_num': '1'}, {'Primary_intent': 'Equipment Support', 'Primary_intent_detail': 'PnP', 'cont_display_metric_name': 'PnP_containment_rate', 'cont_metric_seq_num': '2'}]}, {'feature_name': 'BIL LING', 'feature_id': '1', 'metrics': [{'Primary_intent': 'Billing', 'Primary_intent_detail': 'Billing Concern', 'cont_display_metric_name': 'Billing_containment_rate', 'cont_metric_seq_num': '1'}, {'Primary_intent': 'Billing', 'Primary_intent_detail': 'Billing General', 'cont_display_metric_name': 'Billing_containment_rate', 'cont_metric_seq_num': '2'}, {'Primary_intent': 'Billing', 'Primary_intent_detail': 'Billing Preferences', 'cont_display_metric_name': 'Billing_containment_rate', 'cont_metric_seq_num': '3'}, {'Primary_intent': 'Billing', 'Primary_intent_detail': 'EasyPa

In [5]:
def create_dataframe_from_json(data):
    # Flatten the JSON and create a list of dictionaries
    flattened_data = []

    for feature in data["features"]:
        feature_name = feature["feature_name"]
        feature_id = feature["feature_id"]

        for metric in feature["metrics"]:
            flattened_data.append({
                "hs_feature_name": feature_name,
                "hs_feature_id": feature_id,
                "primary_intent": metric["Primary_intent"],
                "primary_intent_detail": metric["Primary_intent_detail"],
                "cont_display_metric_name": metric["cont_display_metric_name"],
                "cont_metric_seq_num": metric["cont_metric_seq_num"]
            })

    # Create the DataFrame
    df = pd.DataFrame(flattened_data)

    # Add the 'Containment_metric_id' column as a sequence number
    df["containment_metric_id"] = range(1, len(df) + 1)
    df['create_dt'] = pd.to_datetime('today').normalize().date()

    # Reorder the columns if needed
    df = df[["containment_metric_id", "hs_feature_name", "hs_feature_id", "primary_intent", "primary_intent_detail", "cont_display_metric_name", "cont_metric_seq_num", "create_dt"]]

    return df

In [6]:
# Call the function with JSON data
df = create_dataframe_from_json(config_data)

# Print the DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)
print(df.head())

   containment_metric_id hs_feature_name hs_feature_id     primary_intent  \
0                      1      SMART HELP            15  Equipment Support   
1                      2      SMART HELP            15  Equipment Support   
2                      3        BIL LING             1            Billing   
3                      4        BIL LING             1            Billing   
4                      5        BIL LING             1            Billing   

  primary_intent_detail    cont_display_metric_name cont_metric_seq_num  \
0             SmartHelp  Smarthelp_containment_rate                   1   
1                   PnP        PnP_containment_rate                   2   
2       Billing Concern    Billing_containment_rate                   1   
3       Billing General    Billing_containment_rate                   2   
4   Billing Preferences    Billing_containment_rate                   3   

    create_dt  
0  2025-02-18  
1  2025-02-18  
2  2025-02-18  
3  2025-02-18  
4  202

In [7]:
first_table_spark_df = spark.createDataFrame(df)
first_table_spark_df.show()

+---------------------+---------------+-------------+-----------------+---------------------+------------------------+-------------------+----------+
|containment_metric_id|hs_feature_name|hs_feature_id|   primary_intent|primary_intent_detail|cont_display_metric_name|cont_metric_seq_num| create_dt|
+---------------------+---------------+-------------+-----------------+---------------------+------------------------+-------------------+----------+
|                    1|     SMART HELP|           15|Equipment Support|            SmartHelp|    Smarthelp_contain...|                  1|2025-02-18|
|                    2|     SMART HELP|           15|Equipment Support|                  PnP|    PnP_containment_rate|                  2|2025-02-18|
|                    3|       BIL LING|            1|          Billing|      Billing Concern|    Billing_containme...|                  1|2025-02-18|
|                    4|       BIL LING|            1|          Billing|      Billing General|    Bil

In [8]:
first_table_spark_df.printSchema()

root
 |-- containment_metric_id: long (nullable = true)
 |-- hs_feature_name: string (nullable = true)
 |-- hs_feature_id: string (nullable = true)
 |-- primary_intent: string (nullable = true)
 |-- primary_intent_detail: string (nullable = true)
 |-- cont_display_metric_name: string (nullable = true)
 |-- cont_metric_seq_num: string (nullable = true)
 |-- create_dt: date (nullable = true)


In [9]:
from pyspark.sql.functions import col

# Cast the columns to integer
first_table_spark_df_transformed = first_table_spark_df.withColumn("containment_metric_id", col("containment_metric_id").cast("int")) \
       .withColumn("hs_feature_id", col("hs_feature_id").cast("int")) \
       .withColumn("cont_metric_seq_num", col("cont_metric_seq_num").cast("int"))

# Show the schema to verify the changes
first_table_spark_df_transformed.printSchema()


root
 |-- containment_metric_id: integer (nullable = true)
 |-- hs_feature_name: string (nullable = true)
 |-- hs_feature_id: integer (nullable = true)
 |-- primary_intent: string (nullable = true)
 |-- primary_intent_detail: string (nullable = true)
 |-- cont_display_metric_name: string (nullable = true)
 |-- cont_metric_seq_num: integer (nullable = true)
 |-- create_dt: date (nullable = true)


In [10]:
first_output_path = "s3://cci-dig-aicoe-data-sb/processed/containment_metric/containment_metric_master/"
firsttable_writedf = first_table_spark_df_transformed.coalesce(1).write.format("parquet").mode("append").save(first_output_path)

In [11]:
# Get unique values from the columns
primary_intent_values = df['primary_intent'].unique()
primary_intent_detail_values = df['primary_intent_detail'].unique()

In [12]:
print(primary_intent_values)
print(primary_intent_detail_values)

['Equipment Support' 'Billing']
['SmartHelp' 'PnP' 'Billing Concern' 'Billing General'
 'Billing Preferences' 'EasyPay' 'Non-Pay Suspension/Disconnect'
 'Pay Bill' 'Payment Arrangement' 'Payment Concerns' 'Payment Method'
 'Statement Inquiry' 'Usage Inquiry']


In [13]:
# Convert arrays to strings formatted for SQL IN clauses
primary_intent_str = "', '".join(primary_intent_values)
primary_intent_detail_str = "', '".join(primary_intent_detail_values)

In [14]:
print(primary_intent_str)
print(primary_intent_detail_str)

Equipment Support', 'Billing
SmartHelp', 'PnP', 'Billing Concern', 'Billing General', 'Billing Preferences', 'EasyPay', 'Non-Pay Suspension/Disconnect', 'Pay Bill', 'Payment Arrangement', 'Payment Concerns', 'Payment Method', 'Statement Inquiry', 'Usage Inquiry


In [15]:
second_table_query = f"""
SELECT 
    primary_intent, primary_intent_detail,initial_channel,lob,  
    CAST(contact_dt AS DATE) AS contact_dt, 
    COUNT(DISTINCT sub_contact_id) AS sub_contact_id, 
    COUNT(DISTINCT CASE WHEN selfservice_containment = 1 THEN sub_contact_id END) AS selfservice_containment,
    CASE 
        WHEN COUNT(DISTINCT sub_contact_id) > 0 THEN
            ROUND(CAST(SUM(CASE WHEN selfservice_containment = 1 THEN 1 ELSE 0 END) AS DOUBLE) 
            / COUNT(DISTINCT sub_contact_id) * 100, 2)
        ELSE
            0
    END AS containment_rate
FROM 
    ota_data_assets_temp.omni_intent_cntct_fact 
WHERE 
    CAST(contact_dt AS DATE) BETWEEN 
        date_add((SELECT max(CAST(contact_dt AS DATE)) FROM ota_data_assets_temp.omni_intent_cntct_fact), -60) 
        AND (SELECT max(CAST(contact_dt AS DATE)) FROM ota_data_assets_temp.omni_intent_cntct_fact)
    AND primary_intent IN ('{primary_intent_str}')
    AND initial_channel = 'CoxApp'
    AND lob = 'R'
    AND primary_intent_detail IN ('{primary_intent_detail_str}')
GROUP BY 
    primary_intent, primary_intent_detail, contact_dt,initial_channel,lob
ORDER BY 
    contact_dt DESC
"""

In [16]:
second_table_df = spark.sql(second_table_query)
second_table_df.show()

+-----------------+---------------------+---------------+---+----------+--------------+-----------------------+----------------+
|   primary_intent|primary_intent_detail|initial_channel|lob|contact_dt|sub_contact_id|selfservice_containment|containment_rate|
+-----------------+---------------------+---------------+---+----------+--------------+-----------------------+----------------+
|          Billing|      Billing General|         CoxApp|  R|2024-11-02|         19977|                  19944|           99.83|
|          Billing|  Payment Arrangement|         CoxApp|  R|2024-11-02|          2672|                   2633|           98.54|
|Equipment Support|            SmartHelp|         CoxApp|  R|2024-11-02|          1495|                   1386|           92.71|
|          Billing|  Billing Preferences|         CoxApp|  R|2024-11-02|           287|                    280|           97.56|
|          Billing|             Pay Bill|         CoxApp|  R|2024-11-02|         24631|          

In [17]:
second_table_df.printSchema()

root
 |-- primary_intent: string (nullable = true)
 |-- primary_intent_detail: string (nullable = true)
 |-- initial_channel: string (nullable = true)
 |-- lob: string (nullable = true)
 |-- contact_dt: date (nullable = true)
 |-- sub_contact_id: long (nullable = false)
 |-- selfservice_containment: long (nullable = false)
 |-- containment_rate: double (nullable = true)


In [18]:
second_table_df.count()

769


In [19]:
# Assuming you have a Spark DataFrame called spark_df
second_table_pandas_df = second_table_df.toPandas()


In [20]:
second_table_pandas_df.head()

  primary_intent primary_intent_detail initial_channel lob  contact_dt  \
0        Billing     Statement Inquiry          CoxApp   R  2024-11-02   
1        Billing   Payment Arrangement          CoxApp   R  2024-11-02   
2        Billing       Billing General          CoxApp   R  2024-11-02   
3        Billing       Billing Concern          CoxApp   R  2024-11-02   
4        Billing         Usage Inquiry          CoxApp   R  2024-11-02   

   sub_contact_id  selfservice_containment  containment_rate  
0            1437                     1428             99.37  
1            2672                     2633             98.54  
2           19977                    19944             99.83  
3            1218                     1185             97.29  
4           10226                    10187             99.62


In [21]:
# Merge the two dataframes on 'primary_intent_detail' using a left join
combined_df = pd.merge(second_table_pandas_df, df, on='primary_intent_detail', how='left')
combined_df.head()

  primary_intent_x primary_intent_detail initial_channel lob  contact_dt  \
0          Billing     Statement Inquiry          CoxApp   R  2024-11-02   
1          Billing   Payment Arrangement          CoxApp   R  2024-11-02   
2          Billing       Billing General          CoxApp   R  2024-11-02   
3          Billing       Billing Concern          CoxApp   R  2024-11-02   
4          Billing         Usage Inquiry          CoxApp   R  2024-11-02   

   sub_contact_id  selfservice_containment  containment_rate  \
0            1437                     1428             99.37   
1            2672                     2633             98.54   
2           19977                    19944             99.83   
3            1218                     1185             97.29   
4           10226                    10187             99.62   

   containment_metric_id hs_feature_name hs_feature_id primary_intent_y  \
0                     12        BIL LING             1          Billing   
1       

In [22]:
combined_df.columns

Index(['primary_intent_x', 'primary_intent_detail', 'initial_channel', 'lob',
       'contact_dt', 'sub_contact_id', 'selfservice_containment',
       'containment_rate', 'containment_metric_id', 'hs_feature_name',
       'hs_feature_id', 'primary_intent_y', 'cont_display_metric_name',
       'cont_metric_seq_num', 'create_dt'],
      dtype='object')


In [23]:
second_table_columns = ['containment_metric_id', 'primary_intent_x', 'primary_intent_detail', 'sub_contact_id', 'selfservice_containment', 'initial_channel', 'lob', 'contact_dt', 'containment_rate', 'hs_feature_name', 'cont_display_metric_name','create_dt']
second_tabledf = combined_df[second_table_columns]
second_tabledf.head()

   containment_metric_id primary_intent_x primary_intent_detail  \
0                     12          Billing     Statement Inquiry   
1                      9          Billing   Payment Arrangement   
2                      4          Billing       Billing General   
3                      3          Billing       Billing Concern   
4                     13          Billing         Usage Inquiry   

   sub_contact_id  selfservice_containment initial_channel lob  contact_dt  \
0            1437                     1428          CoxApp   R  2024-11-02   
1            2672                     2633          CoxApp   R  2024-11-02   
2           19977                    19944          CoxApp   R  2024-11-02   
3            1218                     1185          CoxApp   R  2024-11-02   
4           10226                    10187          CoxApp   R  2024-11-02   

   containment_rate hs_feature_name  cont_display_metric_name   create_dt  
0             99.37        BIL LING  Billing_contain

In [24]:
second_table_spark_df = spark.createDataFrame(second_tabledf)
second_table_spark_df.show()

+---------------------+-----------------+---------------------+--------------+-----------------------+---------------+---+----------+----------------+---------------+------------------------+----------+
|containment_metric_id| primary_intent_x|primary_intent_detail|sub_contact_id|selfservice_containment|initial_channel|lob|contact_dt|containment_rate|hs_feature_name|cont_display_metric_name| create_dt|
+---------------------+-----------------+---------------------+--------------+-----------------------+---------------+---+----------+----------------+---------------+------------------------+----------+
|                   12|          Billing|    Statement Inquiry|          1437|                   1428|         CoxApp|  R|2024-11-02|           99.37|       BIL LING|    Billing_containme...|2025-02-18|
|                    9|          Billing|  Payment Arrangement|          2672|                   2633|         CoxApp|  R|2024-11-02|           98.54|       BIL LING|    Billing_containme.

In [25]:
second_table_spark_df.printSchema()

root
 |-- containment_metric_id: long (nullable = true)
 |-- primary_intent_x: string (nullable = true)
 |-- primary_intent_detail: string (nullable = true)
 |-- sub_contact_id: long (nullable = true)
 |-- selfservice_containment: long (nullable = true)
 |-- initial_channel: string (nullable = true)
 |-- lob: string (nullable = true)
 |-- contact_dt: date (nullable = true)
 |-- containment_rate: double (nullable = true)
 |-- hs_feature_name: string (nullable = true)
 |-- cont_display_metric_name: string (nullable = true)
 |-- create_dt: date (nullable = true)


In [26]:
second_table_spark_df.count()

830


In [27]:
from pyspark.sql.functions import col

# Change column types and rename columns
second_table_spark_df_transformed = second_table_spark_df \
    .withColumn("containment_metric_id", col("containment_metric_id").cast("int")) \
    .withColumn("sub_contact_id", col("sub_contact_id").cast("int")) \
    .withColumn("selfservice_containment", col("selfservice_containment").cast("int")) \
    .withColumnRenamed("primary_intent_x", "primary_intent")

# Show the transformed dataframe
second_table_spark_df_transformed.show()


+---------------------+-----------------+---------------------+--------------+-----------------------+---------------+---+----------+----------------+---------------+------------------------+----------+
|containment_metric_id|   primary_intent|primary_intent_detail|sub_contact_id|selfservice_containment|initial_channel|lob|contact_dt|containment_rate|hs_feature_name|cont_display_metric_name| create_dt|
+---------------------+-----------------+---------------------+--------------+-----------------------+---------------+---+----------+----------------+---------------+------------------------+----------+
|                   12|          Billing|    Statement Inquiry|          1437|                   1428|         CoxApp|  R|2024-11-02|           99.37|       BIL LING|    Billing_containme...|2025-02-18|
|                    9|          Billing|  Payment Arrangement|          2672|                   2633|         CoxApp|  R|2024-11-02|           98.54|       BIL LING|    Billing_containme.

In [28]:
second_table_spark_df_transformed.printSchema()

root
 |-- containment_metric_id: integer (nullable = true)
 |-- primary_intent: string (nullable = true)
 |-- primary_intent_detail: string (nullable = true)
 |-- sub_contact_id: integer (nullable = true)
 |-- selfservice_containment: integer (nullable = true)
 |-- initial_channel: string (nullable = true)
 |-- lob: string (nullable = true)
 |-- contact_dt: date (nullable = true)
 |-- containment_rate: double (nullable = true)
 |-- hs_feature_name: string (nullable = true)
 |-- cont_display_metric_name: string (nullable = true)
 |-- create_dt: date (nullable = true)


In [29]:
second_output_path = "s3://cci-dig-aicoe-data-sb/processed/containment_metric/containment_metric_data/"
secondtable_writedf = second_table_spark_df_transformed.coalesce(1).write.format("parquet").mode("append").save(second_output_path)

In [30]:
second_tabledf

     containment_metric_id   primary_intent_x          primary_intent_detail  \
0                       12            Billing              Statement Inquiry   
1                        9            Billing            Payment Arrangement   
2                        4            Billing                Billing General   
3                        3            Billing                Billing Concern   
4                       13            Billing                  Usage Inquiry   
..                     ...                ...                            ...   
825                      4            Billing                Billing General   
826                      1  Equipment Support                      SmartHelp   
827                      7            Billing  Non-Pay Suspension/Disconnect   
828                      5            Billing            Billing Preferences   
829                     10            Billing               Payment Concerns   

     sub_contact_id  selfservice_contai

In [31]:
third_table_columns = ['containment_metric_id', 'primary_intent_x', 'primary_intent_detail', 'contact_dt', 'containment_rate']
third_tabledf = second_tabledf[third_table_columns]
third_tabledf.head()

   containment_metric_id primary_intent_x primary_intent_detail  contact_dt  \
0                     12          Billing     Statement Inquiry  2024-11-02   
1                      9          Billing   Payment Arrangement  2024-11-02   
2                      4          Billing       Billing General  2024-11-02   
3                      3          Billing       Billing Concern  2024-11-02   
4                     13          Billing         Usage Inquiry  2024-11-02   

   containment_rate  
0             99.37  
1             98.54  
2             99.83  
3             97.29  
4             99.62


In [32]:
# Pivot the data
df_pivot = third_tabledf.pivot_table(index=['containment_metric_id', 'primary_intent_x', 'primary_intent_detail'],
                          columns='contact_dt',
                          values='containment_rate',
                          aggfunc='first').reset_index()

# Display the result
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 200)      # Show 100 rows
df_pivot.head()

contact_dt  containment_metric_id   primary_intent_x primary_intent_detail  \
0                               1  Equipment Support             SmartHelp   
1                               2  Equipment Support                   PnP   
2                               3            Billing       Billing Concern   
3                               4            Billing       Billing General   
4                               5            Billing   Billing Preferences   

contact_dt  2024-09-03  2024-09-04  2024-09-05  2024-09-06  2024-09-07  \
0                93.17       93.33       92.69       93.31       93.77   
1                89.20       87.94       89.02       89.53       89.94   
2                82.52       84.95       83.70       86.07       97.69   
3                99.45       99.58       99.40       99.50       99.80   
4                98.42       97.34       97.06       97.43       99.12   

contact_dt  2024-09-08  2024-09-09  2024-09-10  2024-09-11  2024-09-12  \
0           

In [33]:
print(len(df_pivot.columns))

64


In [34]:
df_pivot.dtypes

contact_dt
containment_metric_id      int64
primary_intent_x          object
primary_intent_detail     object
2024-09-03               float64
2024-09-04               float64
2024-09-05               float64
2024-09-06               float64
2024-09-07               float64
2024-09-08               float64
2024-09-09               float64
2024-09-10               float64
2024-09-11               float64
2024-09-12               float64
2024-09-13               float64
2024-09-14               float64
2024-09-15               float64
2024-09-16               float64
2024-09-17               float64
2024-09-18               float64
2024-09-19               float64
2024-09-20               float64
2024-09-21               float64
2024-09-22               float64
2024-09-23               float64
2024-09-24               float64
2024-09-25               float64
2024-09-26               float64
2024-09-27               float64
2024-09-28               float64
2024-09-29               float64

In [35]:
print(df_pivot.head())

contact_dt  containment_metric_id   primary_intent_x primary_intent_detail  \
0                               1  Equipment Support             SmartHelp   
1                               2  Equipment Support                   PnP   
2                               3            Billing       Billing Concern   
3                               4            Billing       Billing General   
4                               5            Billing   Billing Preferences   

contact_dt  2024-09-03  2024-09-04  2024-09-05  2024-09-06  2024-09-07  \
0                93.17       93.33       92.69       93.31       93.77   
1                89.20       87.94       89.02       89.53       89.94   
2                82.52       84.95       83.70       86.07       97.69   
3                99.45       99.58       99.40       99.50       99.80   
4                98.42       97.34       97.06       97.43       99.12   

contact_dt  2024-09-08  2024-09-09  2024-09-10  2024-09-11  2024-09-12  \
0           

In [36]:
df_pivot.dtypes

contact_dt
containment_metric_id      int64
primary_intent_x          object
primary_intent_detail     object
2024-09-03               float64
2024-09-04               float64
2024-09-05               float64
2024-09-06               float64
2024-09-07               float64
2024-09-08               float64
2024-09-09               float64
2024-09-10               float64
2024-09-11               float64
2024-09-12               float64
2024-09-13               float64
2024-09-14               float64
2024-09-15               float64
2024-09-16               float64
2024-09-17               float64
2024-09-18               float64
2024-09-19               float64
2024-09-20               float64
2024-09-21               float64
2024-09-22               float64
2024-09-23               float64
2024-09-24               float64
2024-09-25               float64
2024-09-26               float64
2024-09-27               float64
2024-09-28               float64
2024-09-29               float64

In [41]:
import numpy as np
import pandas as pd

def calculate_last7_and_30_days(df):
    # Check the actual columns in the dataframe to ensure we are extracting date columns correctly
    print("Columns in df:", df.columns)

    # Try to detect the date columns using the column names
    datecolumn = []
    for col in df.columns:
        try:
            # Try converting the column name to a date if it's a string
            pd.to_datetime(col, format='%Y-%m-%d', errors='raise')
            datecolumn.append(col)
        except (ValueError, TypeError):
            # Ignore columns that cannot be converted to datetime
            continue
    
    print("Date columns detected:", datecolumn)

    if not datecolumn:
        raise ValueError("No valid date columns found in the dataframe.")

    # Exclude the latest date column (the first one)
    last_7_columns = datecolumn[-8:-1]  # Get the last 7 columns excluding the latest date
    last_30_columns = datecolumn[-31:-1]
    print("Last 7 columns:", last_7_columns)
    print("Last 30 columns:", last_30_columns)

    # Convert the date columns to datetime objects
    date_objects = [pd.to_datetime(col, format='%Y-%m-%d') for col in datecolumn]
    print("Date objects:", date_objects)

    # Get the column corresponding to the latest date
    latest_date = max(date_objects)
    latest_date_column = datecolumn[date_objects.index(latest_date)]

    # Create a new column 'Yesterday' with the values from the latest date column
    df.loc[:, 'Yesterday'] = df[latest_date_column]
    
    # Ensure numeric columns before performing mean calculation
    df[last_7_columns] = df[last_7_columns].apply(pd.to_numeric, errors='coerce')
    df[last_30_columns] = df[last_30_columns].apply(pd.to_numeric, errors='coerce')

    # Calculate the mean for each row across the last 7 and 30 date columns
    df.loc[:, 'last_7_days'] = df[last_7_columns].mean(axis=1)
    df.loc[:, 'last_30_days'] = df[last_30_columns].mean(axis=1)

    # Replace 0 and NaN values in 'last_7_days' and 'last_30_days' with NaN to avoid division by zero
    df['last_7_days'] = df['last_7_days'].replace(0, np.nan)
    df['last_30_days'] = df['last_30_days'].replace(0, np.nan)

    # Calculate the percentage change for 'Last 7 Days' with a check for NaN
    df.loc[:, '% Change Last 7 Days'] = np.where(
        df['last_7_days'].isna(), 0, 
        (df['Yesterday'] - df['last_7_days']) / df['last_7_days'] * 100
    )

    # Calculate the percentage change for 'Last 30 Days' with a check for NaN
    df.loc[:, '% Change Last 30 Days'] = np.where(
        df['last_30_days'].isna(), 0, 
        (df['Yesterday'] - df['last_30_days']) / df['last_30_days'] * 100
    )

    # Round the percentage changes to 1 decimal place
    df.loc[:, '% Change Last 7 Days'] = df['% Change Last 7 Days'].round(1)
    df.loc[:, '% Change Last 30 Days'] = df['% Change Last 30 Days'].round(1)
    
    return df

In [45]:
last_7_and_30_days_df = calculate_last7_and_30_days(df_pivot)
print(last_7_and_30_days_df)

Columns in df: Index(['containment_metric_id',      'primary_intent_x',
       'primary_intent_detail',              2024-09-03,
                    2024-09-04,              2024-09-05,
                    2024-09-06,              2024-09-07,
                    2024-09-08,              2024-09-09,
                    2024-09-10,              2024-09-11,
                    2024-09-12,              2024-09-13,
                    2024-09-14,              2024-09-15,
                    2024-09-16,              2024-09-17,
                    2024-09-18,              2024-09-19,
                    2024-09-20,              2024-09-21,
                    2024-09-22,              2024-09-23,
                    2024-09-24,              2024-09-25,
                    2024-09-26,              2024-09-27,
                    2024-09-28,              2024-09-29,
                    2024-09-30,              2024-10-01,
                    2024-10-02,              2024-10-03,
                

In [42]:
# Check the columns in your df to identify potential issues
print("Columns in df_pivot:", df_pivot.columns)

# Ensure columns like 'primary_intent' exist
expected_columns = ['containment_metric_id', 'primary_intent', 'primary_intent_detail']
missing_columns = [col for col in expected_columns if col not in df_pivot.columns]

if missing_columns:
    print(f"Warning: The following expected columns are missing: {missing_columns}")
else:
    # Example of applying the function
    last_7_and_30_days_df = calculate_last7_and_30_days(df_pivot)
    print(last_7_and_30_days_df)

    #last_7_and_30_days_df = last_7_and_30_days_df[['containment_metric_id', 'primary_intent', 'primary_intent_detail', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']]
    #pd.set_option('display.max_columns', None)  # Show all columns
    #pd.set_option('display.max_rows', 100)
    #print(last_7_and_30_days_df.head())

Columns in df_pivot: Index(['containment_metric_id',      'primary_intent_x',
       'primary_intent_detail',              2024-09-03,
                    2024-09-04,              2024-09-05,
                    2024-09-06,              2024-09-07,
                    2024-09-08,              2024-09-09,
                    2024-09-10,              2024-09-11,
                    2024-09-12,              2024-09-13,
                    2024-09-14,              2024-09-15,
                    2024-09-16,              2024-09-17,
                    2024-09-18,              2024-09-19,
                    2024-09-20,              2024-09-21,
                    2024-09-22,              2024-09-23,
                    2024-09-24,              2024-09-25,
                    2024-09-26,              2024-09-27,
                    2024-09-28,              2024-09-29,
                    2024-09-30,              2024-10-01,
                    2024-10-02,              2024-10-03,
          

In [46]:
last_7_and_30_days_df.head()

contact_dt  containment_metric_id   primary_intent_x primary_intent_detail  \
0                               1  Equipment Support             SmartHelp   
1                               2  Equipment Support                   PnP   
2                               3            Billing       Billing Concern   
3                               4            Billing       Billing General   
4                               5            Billing   Billing Preferences   

contact_dt  2024-09-03  2024-09-04  2024-09-05  2024-09-06  2024-09-07  \
0                93.17       93.33       92.69       93.31       93.77   
1                89.20       87.94       89.02       89.53       89.94   
2                82.52       84.95       83.70       86.07       97.69   
3                99.45       99.58       99.40       99.50       99.80   
4                98.42       97.34       97.06       97.43       99.12   

contact_dt  2024-09-08  2024-09-09  2024-09-10  2024-09-11  2024-09-12  \
0           

In [47]:
last_7_and_30_days_df = last_7_and_30_days_df[['containment_metric_id', 'primary_intent_x', 'primary_intent_detail', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)
last_7_and_30_days_df.head()

contact_dt  containment_metric_id   primary_intent_x primary_intent_detail  \
0                               1  Equipment Support             SmartHelp   
1                               2  Equipment Support                   PnP   
2                               3            Billing       Billing Concern   
3                               4            Billing       Billing General   
4                               5            Billing   Billing Preferences   

contact_dt  Yesterday  last_7_days  last_30_days  % Change Last 7 Days  \
0               92.71    94.124286     94.232667                  -1.5   
1               85.00    86.857143     87.695667                  -2.1   
2               97.29    87.427143     87.650000                  11.3   
3               99.83    99.622857     99.613667                   0.2   
4               97.56    98.098571     97.820333                  -0.5   

contact_dt  % Change Last 30 Days  
0                            -1.6  
1             

In [48]:
result_df = pd.merge(df, last_7_and_30_days_df, on="containment_metric_id", how="inner")
result_df.head()

   containment_metric_id hs_feature_name hs_feature_id     primary_intent  \
0                      1      SMART HELP            15  Equipment Support   
1                      2      SMART HELP            15  Equipment Support   
2                      3        BIL LING             1            Billing   
3                      4        BIL LING             1            Billing   
4                      5        BIL LING             1            Billing   

  primary_intent_detail_x    cont_display_metric_name cont_metric_seq_num  \
0               SmartHelp  Smarthelp_containment_rate                   1   
1                     PnP        PnP_containment_rate                   2   
2         Billing Concern    Billing_containment_rate                   1   
3         Billing General    Billing_containment_rate                   2   
4     Billing Preferences    Billing_containment_rate                   3   

    create_dt   primary_intent_x primary_intent_detail_y  Yesterday  \
0  

In [49]:
result_df = result_df[['containment_metric_id', 'hs_feature_name', 'cont_display_metric_name', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days', 'create_dt']]
result_df.head()

   containment_metric_id hs_feature_name    cont_display_metric_name  \
0                      1      SMART HELP  Smarthelp_containment_rate   
1                      2      SMART HELP        PnP_containment_rate   
2                      3        BIL LING    Billing_containment_rate   
3                      4        BIL LING    Billing_containment_rate   
4                      5        BIL LING    Billing_containment_rate   

   Yesterday  last_7_days  last_30_days  % Change Last 7 Days  \
0      92.71    94.124286     94.232667                  -1.5   
1      85.00    86.857143     87.695667                  -2.1   
2      97.29    87.427143     87.650000                  11.3   
3      99.83    99.622857     99.613667                   0.2   
4      97.56    98.098571     97.820333                  -0.5   

   % Change Last 30 Days   create_dt  
0                   -1.6  2025-02-18  
1                   -3.1  2025-02-18  
2                   11.0  2025-02-18  
3                   

In [50]:
result_df[[ 'last_7_days', 'last_30_days']] = result_df[[ 'last_7_days', 'last_30_days']].round(3)

In [51]:
result_df.head()

   containment_metric_id hs_feature_name    cont_display_metric_name  \
0                      1      SMART HELP  Smarthelp_containment_rate   
1                      2      SMART HELP        PnP_containment_rate   
2                      3        BIL LING    Billing_containment_rate   
3                      4        BIL LING    Billing_containment_rate   
4                      5        BIL LING    Billing_containment_rate   

   Yesterday  last_7_days  last_30_days  % Change Last 7 Days  \
0      92.71       94.124        94.233                  -1.5   
1      85.00       86.857        87.696                  -2.1   
2      97.29       87.427        87.650                  11.3   
3      99.83       99.623        99.614                   0.2   
4      97.56       98.099        97.820                  -0.5   

   % Change Last 30 Days   create_dt  
0                   -1.6  2025-02-18  
1                   -3.1  2025-02-18  
2                   11.0  2025-02-18  
3                   

In [52]:
third_table_spark_df = spark.createDataFrame(result_df)
third_table_spark_df.show()

+---------------------+---------------+------------------------+---------+-----------+------------+--------------------+---------------------+----------+
|containment_metric_id|hs_feature_name|cont_display_metric_name|Yesterday|last_7_days|last_30_days|% Change Last 7 Days|% Change Last 30 Days| create_dt|
+---------------------+---------------+------------------------+---------+-----------+------------+--------------------+---------------------+----------+
|                    1|     SMART HELP|    Smarthelp_contain...|    92.71|     94.124|      94.233|                -1.5|                 -1.6|2025-02-18|
|                    2|     SMART HELP|    PnP_containment_rate|     85.0|     86.857|      87.696|                -2.1|                 -3.1|2025-02-18|
|                    3|       BIL LING|    Billing_containme...|    97.29|     87.427|       87.65|                11.3|                 11.0|2025-02-18|
|                    4|       BIL LING|    Billing_containme...|    99.83|  

In [53]:
third_table_spark_df.printSchema()

root
 |-- containment_metric_id: long (nullable = true)
 |-- hs_feature_name: string (nullable = true)
 |-- cont_display_metric_name: string (nullable = true)
 |-- Yesterday: double (nullable = true)
 |-- last_7_days: double (nullable = true)
 |-- last_30_days: double (nullable = true)
 |-- % Change Last 7 Days: double (nullable = true)
 |-- % Change Last 30 Days: double (nullable = true)
 |-- create_dt: date (nullable = true)


In [54]:
from pyspark.sql.functions import col

# Change the data type of 'containment_metric_id' from long to integer
third_table_spark_df = third_table_spark_df.withColumn(
    "containment_metric_id", col("containment_metric_id").cast("int")
)

# Show the schema to confirm the change
third_table_spark_df.printSchema()


root
 |-- containment_metric_id: integer (nullable = true)
 |-- hs_feature_name: string (nullable = true)
 |-- cont_display_metric_name: string (nullable = true)
 |-- Yesterday: double (nullable = true)
 |-- last_7_days: double (nullable = true)
 |-- last_30_days: double (nullable = true)
 |-- % Change Last 7 Days: double (nullable = true)
 |-- % Change Last 30 Days: double (nullable = true)
 |-- create_dt: date (nullable = true)


In [55]:
third_output_path = "s3://cci-dig-aicoe-data-sb/processed/containment_metric/containment_metric_calculation/"
third_df_table_write_df = third_table_spark_df.coalesce(1).write.format("parquet").mode("append").save(third_output_path)